## Philip Uit
### MSDE 621 _ Week 2 Assignment 
### 7/12/20

## Week 2 Assignment

## Part 1
As suggested in Use Case 1 of the FTE, we need to read the other nine stock files and insert their data into the database. While nine files isn't an undue burden to manually read, we are going to look ahead to the time when we may have 100 log files to read and implement this code using the DRY Principle. DRY stands for

Don't
Repeat
Yourself
So, while we could create an individual cell to read each stock file, for this assignment, do it in one. I'll give you some help to get started.

Reference:
https://en.wikipedia.org/wiki/Don%27t_repeat_yourself

Much of computer science and programming is about identifying and exploiting patterns. In this case, we know there is a pattern to how the files are named, and we know (through inspection) that there is a pattern to the columns of data inside.

The file names all start with a year, from 2009 straight to 2019 with no breaks:

2009_aapl_data.xlsx
2010_aapl_data.xlsx
2011_aapl_data.xlsx
2012_aapl_data.xlsx
2013_aapl_data.xlsx
2014_aapl_data.xlsx
2015_aapl_data.xlsx
2016_aapl_data.xlsx
2017_aapl_data.xlsx
2018_aapl_data.xlsx
2019_aapl_data.xlsx
Do we know of anything in Python capable of generating a range of numbers like that?

for x in range(2009, 2020):
    print(x)
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
That should be enough to get you going. Feel free to use code and helper functions from the FTE. I gave you the .ipynb file for a reason :).

Remember,

If you ran the FTE code, it will have already read in 2009 and created the database file.
If you start at 2009, depending on the condition above, you may need to handle the table already existing.
It is your choice whether or not to use the dataset library. As with many things in life, there are tradeoffs -- some things are easier, some not.

### Deliverable:

When you are done, you will have a database table with slightly more than 2500 rows in it. Show this by doing a query that counts rows in the table.
import dataset
from openpyxl import load_workbook

#### Setup:

In [8]:
#importing packages we will use throughout this exercises:

import dataset
from openpyxl import load_workbook
import sqlite3
import glob
import pandas as pd
import numpy as np

Using pattern in file names 2009-2019 to pull all files in at once 

In [11]:
# getting data file names of all files 2009-2019 of aapl stock price data

#using glob.glob funciton to pull all files in the naming pattern 2009-2019.
filenames = glob.glob('*_aapl_data.xlsx')

dfs = []
for filename in filenames:
    dfs.append(pd.read_excel(filename)) 

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)

In [12]:
# checking that all data is pulled into 1 table or dataframe
# we see 2,516 rows with 6 columns. Alse we see 2009 data on top and 2019 data on the bottom. looks good to move on.
df

,date,close,volume,open,high,low
0,2009/12/31,30.1046,87907426.0,30.4471,30.4786,30.0800
1,2009/12/30,30.2343,102705781.0,29.8328,30.2857,29.7586
2,2009/12/29,29.8714,110755363.0,30.3757,30.3886,29.8186
3,2009/12/28,30.2300,160784168.0,30.2457,30.5643,29.9444
4,2009/12/24,29.8628,125222058.0,29.0786,29.9071,29.0500
...,...,...,...,...,...,...
2511,2019/01/08,150.7500,40622910.0,149.5600,151.8200,148.5200
2512,2019/01/07,147.9300,54571440.0,148.7000,148.8300,145.9000
2513,2019/01/04,148.2600,57423650.0,144.5300,148.5499,143.8000
2514,2019/01/03,142.1900,91106840.0,143.9800,145.7200,142.0000


### When you are done, you will have a database table with slightly more than 2500 rows in it. Show this by doing a query that counts rows in the table.
we see in the cell below by checking the length, we see 2,516 rows in the table with all data pulled in. Same as we output in the cell above. Looks good.
### there are 2,516 rows in the table

In [13]:
#query the counts rows in table using length function
len(df)

2516

#### Part 2 setup
checking type and setting up for part 2: 
due to taking a slightly differnt approach on this lab using glob.glob to pull in data. I wanted to check object types, especially for 'date', as this is an important column for part 2 working with isolating individual years. 

In [14]:
#checking df info of data types. This is looking aread at part 2 and how we might need to use annual dates. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    2516 non-null   object 
 1   close   2516 non-null   float64
 2   volume  2516 non-null   float64
 3   open    2516 non-null   float64
 4   high    2516 non-null   float64
 5   low     2516 non-null   float64
dtypes: float64(5), object(1)
memory usage: 118.1+ KB


Changing Dtype of 'date' column to datetime64[ns]. This will help for part 2. 

In [16]:
# changing the data type on the 'date' column
df['date'] = pd.to_datetime(df['date'])
df.info()
#we see we changed the data type for date column - this will help with analysis later on
#we now have the data cleaned and can move on for further analysis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2516 non-null   datetime64[ns]
 1   close   2516 non-null   float64       
 2   volume  2516 non-null   float64       
 3   open    2516 non-null   float64       
 4   high    2516 non-null   float64       
 5   low     2516 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 118.1 KB


Let's get a unique look at this data using the .describe() function. We can see some inetersting statistics about this dataset

In [17]:
#using describe() function to better understand this data. We see a unique perpective here.
df.describe()

,close,volume,open,high,low
count,2516.000000,2.516000e+03,2516.000000,2516.000000,2516.000000
mean,103.132211,7.786911e+07,103.127600,104.049273,102.175100
std,51.792948,5.910572e+07,51.761292,52.227741,51.349539
min,24.330000,1.136205e+07,23.896600,24.385700,23.870000
25%,60.898900,3.250788e+07,60.853900,61.318175,60.300025
50%,96.015000,6.056857e+07,96.000000,96.825000,95.054250
75%,139.035000,1.066025e+08,138.875000,139.680000,138.601925
max,232.070000,4.649832e+08,230.780000,233.470000,229.780000


## Part 2 -- #1
Find all days where the stock closed lower than 25.
Print a count of how many
Print the first 5 rows found

In [18]:
# query of all days that closed lower than $25 a share. 
# we see 6 total. ALl in Sept. 2009
df.query('close <= 25')
#we will print the count of this output in the next cell

,date,close,volume,open,high,low
76,2009-09-14,24.8171,80383404.0,24.4043,24.8428,24.3214
77,2009-09-11,24.5943,87108026.0,24.7014,24.7400,24.4100
78,2009-09-10,24.6514,122612107.0,24.5800,24.7500,24.4014
79,2009-09-09,24.4486,202624511.0,24.6828,24.9243,24.2428
80,2009-09-08,24.7043,78524974.0,24.7114,24.7343,24.5714
81,2009-09-04,24.3300,93309888.0,23.8966,24.3857,23.8700


#### Print of Count of how many days the stock closed <$25 a share:
#### There are 6 days (cell below)

In [19]:
# Print of Count of how many days the stock closed <$25 a share:
len(df.query('close <= 25'))

6

#### Print the first 5 rows found

In [20]:
# query of first 5 rows found  that closed lower than $25 a share. 
# We see it searches ascending from today's date

df.query('close <= 25').head(5)

,date,close,volume,open,high,low
76,2009-09-14,24.8171,80383404.0,24.4043,24.8428,24.3214
77,2009-09-11,24.5943,87108026.0,24.7014,24.7400,24.4100
78,2009-09-10,24.6514,122612107.0,24.5800,24.7500,24.4014
79,2009-09-09,24.4486,202624511.0,24.6828,24.9243,24.2428
80,2009-09-08,24.7043,78524974.0,24.7114,24.7343,24.5714


## Part 2 -- #2
Find all days in 2017 where the stock closed above 35.
Print a count of how many
Print the last 5 found.

code reference for this cell below: https://kite.com/python/answers/how-to-filter-pandas-dataframe-rows-by-date-in-python

We need to sepereate out the year 2017. So we make 1/1/17 the start_date & 1/31/17 the end_date. This will help answer the questions with the desired queries based on the year 2017, which we now isoloate.

In [21]:
# Here we filter all 2017 days to be able to do analysis on just 2017.
# We see we have 251 rows and 6 columns, we confirm we seperated the data correctly.

start_date = "2017-1-1"
end_date = "2017-12-31"

after_start_date = df["date"] >= start_date
before_end_date = df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = df.loc[between_two_dates]

print(filtered_dates)

           date   close      volume    open      high      low
1844 2017-12-29  169.23  25938760.0  170.52  170.5900  169.220
1845 2017-12-28  171.08  16412270.0  171.00  171.8500  170.480
1846 2017-12-27  170.60  21477380.0  170.10  170.7800  169.710
1847 2017-12-26  170.57  33113340.0  170.80  171.4700  169.679
1848 2017-12-22  175.01  16339690.0  174.68  175.4240  174.500
...         ...     ...         ...     ...       ...      ...
2090 2017-01-09  118.99  33420290.0  117.95  119.4300  117.940
2091 2017-01-06  117.91  31714820.0  116.78  118.1600  116.470
2092 2017-01-05  116.61  22115100.0  115.92  116.8642  115.810
2093 2017-01-04  116.02  21081620.0  115.85  116.5100  115.750
2094 2017-01-03  116.15  28769220.0  115.80  116.3300  114.760

[251 rows x 6 columns]


In [22]:
#putting all 2017 data into it's own dataframe callled df1
df1 = filtered_dates

In [23]:
#query of all days in 2017 that closed greater than $35 a share. 
# It appears that everyday was abover 35$. As we sill see 251 rows and 6 columns
df1.query('close >= 35')

,date,close,volume,open,high,low
1844,2017-12-29,169.23,25938760.0,170.52,170.5900,169.220
1845,2017-12-28,171.08,16412270.0,171.00,171.8500,170.480
1846,2017-12-27,170.60,21477380.0,170.10,170.7800,169.710
1847,2017-12-26,170.57,33113340.0,170.80,171.4700,169.679
1848,2017-12-22,175.01,16339690.0,174.68,175.4240,174.500
...,...,...,...,...,...,...
2090,2017-01-09,118.99,33420290.0,117.95,119.4300,117.940
2091,2017-01-06,117.91,31714820.0,116.78,118.1600,116.470
2092,2017-01-05,116.61,22115100.0,115.92,116.8642,115.810
2093,2017-01-04,116.02,21081620.0,115.85,116.5100,115.750


#### Print a Count of how many days in 2017 the stock closed above 35 dollars a share
we use the length function to help with this shown below. 
#### there are 251 days in 2017 that aapl stock price closed >$35. Looks to be everyday possible that closed above that price!

In [24]:
#printing count of days in 2017 > 35$ a share at market close.
len(df1.query('close >= 35')) 

251

#### Print the last 5 days found in 2017 that closed above 35 dollars a share
in the cell below we see the output. notice that the query found the results with a date furthest from today using the tail(5) function to find the last 5
#### the last 5 days found in 2017 that closed above 35 dollars a share: 1/3/17, 1/4/17, 1/5/17, 1/6/17,  & 1/9/17 

In [25]:
# query of last 5 rows found of 2017 days that closed greater than $35 a share. 
# Again, We see it searches ascending from today's date

df1.query('close >= 35').tail(5)

,date,close,volume,open,high,low
2090,2017-01-09,118.99,33420290.0,117.95,119.4300,117.94
2091,2017-01-06,117.91,31714820.0,116.78,118.1600,116.47
2092,2017-01-05,116.61,22115100.0,115.92,116.8642,115.81
2093,2017-01-04,116.02,21081620.0,115.85,116.5100,115.75
2094,2017-01-03,116.15,28769220.0,115.80,116.3300,114.76


## Setting up and placing data into Database using SQlite

In [26]:
#importing create_engine 
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [27]:
#converting df to sql using to_sql function. We also name it 'aapl' and assign the con engine. 
df.to_sql('aapl', con=engine)

In [28]:
# creating the database with the converted to_sql df from above.
db = dataset.connect("sqlite:///" + 'aapl')

In [30]:
# Use a for loop just in case someone snuck in a new table on us
if (len(db.tables) > 0):
    for table in db.tables:
        db[table].drop()

In [31]:
# creating table 'aapl'
table = db.create_table("aapl")

#### testing new DB and running query in engine.execute to fetchall data from the new DB. We see we have all the data successfull placed into the new 'aapl.db'

In [32]:
# using engine.execute to fetchall data. It looks like it is all there and looks clean. We could do query searches from this using the engine.execute function to search other queries if needed. 
engine.execute("SELECT * FROM aapl").fetchall()

[(0, '2009-12-31 00:00:00.000000', 30.1046, 87907426.0, 30.4471, 30.4786, 30.08),
 (1, '2009-12-30 00:00:00.000000', 30.2343, 102705781.0, 29.8328, 30.2857, 29.7586),
 (2, '2009-12-29 00:00:00.000000', 29.8714, 110755363.0, 30.3757, 30.3886, 29.8186),
 (3, '2009-12-28 00:00:00.000000', 30.23, 160784168.0, 30.2457, 30.5643, 29.9444),
 (4, '2009-12-24 00:00:00.000000', 29.8628, 125222058.0, 29.0786, 29.9071, 29.05),
 (5, '2009-12-23 00:00:00.000000', 28.8714, 86118086.0, 28.75, 28.9114, 28.6871),
 (6, '2009-12-22 00:00:00.000000', 28.6228, 87148416.0, 28.4914, 28.6928, 28.38),
 (7, '2009-12-21 00:00:00.000000', 28.3186, 152166116.0, 28.0071, 28.5357, 27.9528),
 (8, '2009-12-18 00:00:00.000000', 27.9186, 151863506.0, 27.5957, 27.929, 27.5143),
 (9, '2009-12-17 00:00:00.000000', 27.4086, 96720359.0, 27.7514, 27.8571, 27.2857),
 (10, '2009-12-16 00:00:00.000000', 27.8614, 88203036.0, 27.8714, 28.0714, 27.7928),
 (11, '2009-12-15 00:00:00.000000', 27.7386, 104742851.0, 27.9757, 28.2157, 27.6

#### creating aapl.db file

In [33]:
# creating aapl.db and setting as active
data_file = df.to_sql('aapl', con=engine)
workbook = load_workbook(filename=data_file)
sheet = workbook.active
# we see this already exists, as I have ran this command before on my computer. We can at least confirm it is created.

ValueError: Table 'aapl' already exists.

## Summary
In conclusion, we succesfully were able to load the 2009-2019 aapl_data.xslm files into a table. We were able to succesfully load all files with the pattern of the years in order using a slightly differnt method, glob.glob. This method was the cleanest in seting up a table for me in the week of practicing on this dataset, as it made the most sence to use this method for me. We were easily able to pull in all files and read them to a dataframe, which we could run all the desired queries, and also convert into a database at the end to store the data using SQLite. We found a count and printed all rows found when the stock closed lower then 25$, which was only 6 days total. We were able to find all days in 2017 that closed above 35$ a share, and also printed the last 5 found. We successfully converted the table we used into a database using SQLite. We used fetchall to query a search in the DB. We are now setup to do any query we would like. Convientently, we also now have the data in a dataframe with datetime setup for 'date' column and now we also have a database setup with the ability to query and manipulate wither as needed. 

## References:

Big Data file formats. (2020, April 23). Retrieved from https://luminousmen.com/post/big-data-file-formats.

Lawton, G. (2019, February 22). What to consider when choosing big data file formats. In Seach Business Analytics. Retrieved from https://searchbusinessanalytics.techtarget.com/feature/What-to-consider-when-choosing-big-data-file-formats.
